In [2]:
!pip install snowflake-connector-python

^C


In [1]:
import yfinance as yf
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [2]:
# Fetch NVIDIA data
ticker = yf.Ticker("NVDA")
data = ticker.info

In [3]:
valuation_data = {
    "Market Cap (intraday)": data.get("marketCap", "N/A"),
    "Enterprise Value": data.get("enterpriseValue", "N/A"),
    "Trailing P/E": data.get("trailingPE", "N/A"),
    "Forward P/E": data.get("forwardPE", "N/A"),
    "PEG Ratio": data.get("pegRatio", "N/A"),
    "Price/Sales": data.get("priceToSalesTrailing12Months", "N/A"),
    "Enterprise Value/Revenue": data.get("enterpriseToRevenue", "N/A"),
    "Enterprise Value/EBITDA": data.get("enterpriseToEbitda", "N/A")
}

In [4]:
valuation_data

{'Market Cap (intraday)': 2944836042752,
 'Enterprise Value': 2929463918592,
 'Trailing P/E': 41.05102,
 'Forward P/E': 29.293692,
 'PEG Ratio': 'N/A',
 'Price/Sales': 22.56631,
 'Enterprise Value/Revenue': 22.449,
 'Enterprise Value/EBITDA': 35.16}

In [5]:
# Configure Snowflake connection
conn = snowflake.connector.connect(
    user="yohanmarkose",
    password="ymarko@Yo120",
    account="nbb12463.us-east-1",
    warehouse="COMPUTE_WH",
    database="LANGRAPH_DB",
    schema="NVIDIA_SCHEMA"
)

In [6]:
# Insert data
cursor = conn.cursor()
insert_query = """
INSERT INTO NVIDIA_VALUATION (
    MARKET_CAP,
    ENTERPRISE_VALUE,
    TRAILING_PE,
    FORWARD_PE,
    PEG_RATIO,
    PRICE_TO_SALES,
    ENTERPRISE_VALUE_TO_REVENUE,
    ENTERPRISE_VALUE_TO_EBITDA
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

values = (
    valuation_data["Market Cap (intraday)"],
    valuation_data["Enterprise Value"],
    valuation_data["Trailing P/E"],
    valuation_data["Forward P/E"],
    valuation_data["PEG Ratio"],
    valuation_data["Price/Sales"],
    valuation_data["Enterprise Value/Revenue"],
    valuation_data["Enterprise Value/EBITDA"]
)

cursor.execute(insert_query, values)
conn.commit()
cursor.close()
conn.close()

ProgrammingError: 100038 (22018): Numeric value 'N/A' is not recognized